In [1]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0

    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if not same_object_detected:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids


In [54]:
import cv2
import numpy as np 
import time
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

carI ={}
carO = {}
class App(tk.Tk):
    def __init__(self, video_source, total_cars, default_max_speed=60):
        tk.Tk.__init__(self)
        self.title("Marcador de velocidades")

        # Estilo ttk
        style = ttk.Style()
        style.configure('TFrame', background='#ececec')
        style.configure('TLabel', background='#ececec', font=('Arial', 10))
        style.configure('TButton', background='#4caf50', font=('Arial', 10))
        self.count_vehicules = 0 
        self.max_speed = default_max_speed
        self.current_id = None
        self.total_cars = total_cars

        # Contenedor principal
        contenedor_principal = ttk.Frame(self)
        contenedor_principal.pack(side=tk.LEFT, anchor=tk.N)

        # Contenedor para alinear horizontalmente los elementos en la parte izquierda
        contenedor_izquierda = ttk.Frame(contenedor_principal)
        contenedor_izquierda.grid(row=0, column=0, sticky="nsew", padx=5)

        # Etiqueta para "Maxima velocidad permitida"
        self.text_max_speed = ttk.Label(contenedor_izquierda, text="Maxima velocidad permitida")
        self.text_max_speed.grid(row=0, column=0, sticky="w", padx=1)

        # Entrada para el valor
        self.entrada_entero = ttk.Entry(contenedor_izquierda)
        self.entrada_entero.grid(row=0, column=1, sticky="w", padx=5)

        # Botón "Mostrar Valor"
        self.boton_mostrar = ttk.Button(contenedor_izquierda, text="Establecer velocidad máxima", command=self.validar_entero)
        self.boton_mostrar.grid(row=0, column=2, sticky="w", pady=5)

        # Panel de ayuda
        self.ayuda_label = ttk.Label(contenedor_izquierda, text="", foreground='red')
        self.ayuda_label.grid(row=4, column=0, columnspan=2, pady=5)

        self.text_count_vehicules = ttk.Label(contenedor_izquierda, text=f"Numero de vehiculos detectados: {self.count_vehicules}")
        self.text_count_vehicules.grid(row=1, column=0, sticky="w", padx=1)

        self.count_label = ttk.Label(contenedor_izquierda, text=f"Coches con velocidad superior a {self.max_speed}: {self.total_cars}")
        self.count_label.grid(row=2, column=0, sticky="w", padx=1)

        self.btn_quit = ttk.Button(contenedor_izquierda, text="Salir", command=self.quit)
        self.btn_quit.grid(row=3, column=0, sticky="w", padx=1)

        self.deteccion = cv2.createBackgroundSubtractorMOG2(history=10000, varThreshold=100)
        self.cap = cv2.VideoCapture(video_source)

        if not self.cap.isOpened():
            print("Error al abrir el video")
            exit()
        self.tracker = Tracker()
        self.canvas = tk.Canvas(contenedor_principal, width=self.cap.get(cv2.CAP_PROP_FRAME_WIDTH),
                                height=self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.canvas.grid(row=0, column=1, sticky="nsew")

        # Centrar la ventana en la pantalla
        self.update_idletasks()
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        x_position = (screen_width - self.winfo_width()) // 2
        y_position = (screen_height - self.winfo_height()) // 2
        self.geometry(f"+{x_position}+{y_position}")
        
        self.update()
    def update(self):
        ret, frame = self.cap.read()

        if not ret:
            # Si el video ha terminado, volvemos al principio
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            # Reiniciamos las variables de seguimiento
            global carI
            global carO
            carI = {}
            carO = {}

        # Verifica que el fotograma no esté vacío antes de procesarlo
        if frame is not None:
            result_frame = self.process_frame(frame)
            self.photo = self.convert_frame_to_photo(result_frame)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)

        self.after(5, self.update)
            
    def validar_entero(self):
        nuevo_valor = self.entrada_entero.get()
        if nuevo_valor.isdigit() and 0 <= int(nuevo_valor) <= 200:
            # Solo permite dígitos
            self.max_speed = nuevo_valor
            self.count_label.config(text=f"Coches con velocidad superior a {self.max_speed}: {self.total_cars}")
            self.ayuda_label.config(text="")
            # Lógica adicional si es necesario
        else:
            # Muestra un mensaje de ayuda si se introduce algo no permitido
            self.ayuda_label.config(text="Ingrese un número válido entre 0 y 200")


    def process_frame(self, frame):
        global carI  
  

        ret, frame  = self.cap.read()
        if ret == True:

            height= frame.shape[0]
            width= frame.shape[1]

            #Creamos una mascara
            mask = np.zeros((height,width),dtype=np.uint8)
            #Elegimos nuestra zona de interes
            #Elegimos los puntos

            pts = np.array([[[1835,500],[70,500],[70,630],[1835,630]]])
            cv2.fillPoly(mask,pts,255)

            area2 = np.array([[[790,520],[1430,520],[1430,620],[790,620]]])
            area3 = np.array([[[120,520],[780,520],[780,620],[120,620]]])


            cv2.polylines(frame,[np.array(area2,np.int32)], True, (0,0,255),2)
            cv2.polylines(frame,[np.array(area3,np.int32)], True, (255,255,0),2)

            #Eliminarmos lo que esta afuera de la zona
            zona = cv2.bitwise_and(frame,frame,mask=mask)



            #Eliminamos ruido
            mascara = self.deteccion.apply(zona)
            filtro = cv2.GaussianBlur(mascara,(11,11),0)
            #cv2.imshow("zona",filtro)

            #Aplicamos umbralizado
            
            _, umbral = cv2.threshold(filtro,90,255,cv2.THRESH_BINARY)
            
            #Dilatamos los pixeles
            dila = cv2.dilate(umbral,np.ones((3,3)))
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
            cerrar = cv2.morphologyEx(dila,cv2.MORPH_CLOSE, kernel)

            
            contornos, _ = cv2.findContours(cerrar,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            #cv2.imshow("contornos",filtro)
            detecciones = []
            
            #Aplica,os kernel para ajustar los pixeles dispersos
            
            for cont in contornos:
                #Eliminamos contornos pequeño
                area = cv2.contourArea(cont)
                if area > 400:
                    
                    x,y, ancho, alto = cv2.boundingRect(cont)
                    
                    #Guardamos los contornos
                    detecciones.append([x,y,ancho,alto])
            info_id = self.tracker.update(detecciones)
            for inf in info_id:     
                x, y, ancho, alto, id = inf


                #Extraemos el centro
                cx = int(x+ancho / 2)
                cy = int(y + alto / 2)

                #area de influencia
                a2 = cv2.pointPolygonTest(np.array(area2,np.int32),(cx,cy),False)

                #Si nos encontramos en el area 2 

                if a2 >= 0 and id not in carI:
                    carI[id] = time.process_time()
                if id in carI:
                    cv2.circle(frame,(cx,cy),3,(0,0,255),-1)
                    #Si entra en el area 3
                    a3 = cv2.pointPolygonTest(np.array(area3,np.int32),(cx,cy),False)
                    if a3 >= 0:

                        tiempo = time.process_time()- carI[id]

                        if id not in carO:
                            self.current_id = 1
                            carO[id] = tiempo
                            self.count_vehicules = len(carO)
                            self.text_count_vehicules.config(text=f"Numero de vehiculos detectados: {self.count_vehicules}")
                        if id in carO:
                            tiempo = carO[id]
                            vel = 100/carO[id]
                            vel = vel * 3.6
                            if self.current_id == 1 and int(vel) > int(self.max_speed):
                                    self.current_id  = 0
                                    self.total_cars += 1
                                    self.count_label.config(text=f"Coches con velocidad superior a {self.max_speed}: {self.total_cars}")
                            

                        cv2.rectangle(frame,(x,y-10),(x+ancho, y +alto),(0,0,255),2)
                        cv2.putText(frame,str(int(vel))+"KM/H",(x,y-35),cv2.FONT_HERSHEY_PLAIN,3,(255,255,255),2)
                       

        return frame

    def convert_frame_to_photo(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        photo = ImageTk.PhotoImage(image=img)
        return photo

    def quit(self):
        self.cap.release()
        self.destroy()

if __name__ == "__main__":
    app = App(video_source="video.mp4",total_cars = 0)
    app.mainloop()
